In [14]:
from sentence_transformers import SentenceTransformer
import pandas as pd
import numpy as np
from sklearn.cluster import OPTICS, cluster_optics_dbscan

In [15]:
model = SentenceTransformer('sentence-transformers/paraphrase-MiniLM-L3-v2')


In [18]:
data = pd.read_csv("../data/location_data.csv",index_col=None)

In [19]:
data.describe()

,city,road,house_number,postcode,state
count,63695,63695,63695,63695,63695
unique,5534,17184,6353,11015,121
top,houston,main st,101,46227,ca
freq,640,599,357,33,3251


In [20]:
data.head()

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl


In [6]:
data.shape

(63695, 5)

In [30]:
data_no_dup = data.drop_duplicates(ignore_index=True)
data_no_dup.shape # we have many duplicate rows

(41423, 5)

In [31]:
data_no_dup # we're keeping the old indexes

,city,road,house_number,postcode,state
0,fort worth,sycamore school rd,5421,76123,texas
1,gautier,highway 90,2701,39553,mississippi
2,farmington,e main,4750,87401,nm
3,richland,queensgate drive,2751,99352,washington
4,davie,s state hwy seven,1600,33317,fl
...,...,...,...,...,...
41418,highland,baseline st,27774,92346,california
41419,hilliard,main st,3685,43026,ohio
41420,window rock,window rock loop road,264,86515,az
41421,denver,s santa fe dr,1395,80223,co


In [32]:
sentences = data_no_dup["city"]+" "+ data_no_dup["road"]+" "+ data_no_dup["house_number"]\
            +" "+ data_no_dup["postcode"]+" "+ data_no_dup["state"]

In [33]:
sentences[34357] # this doesn't seem to be an address in the USA...

'kota jkt utara jl. boulevard artha gading jalan kav no.01 rt.18/rw.8 klp. gading bar. kec. klp. gading 18 14240 daerah khusus ibukota jakarta'

In [34]:
sentence_embeddings = model.encode(sentences)
# this will not scale well...

In [13]:
pd.DataFrame(sentence_embeddings).to_csv("../data/embeddings.csv")

In [24]:
clustering = OPTICS(min_samples=2, cluster_method = "dbscan",
                    n_jobs = -1, algorithm = "ball_tree").fit(sentence_embeddings.astype("float16"))

KeyboardInterrupt: 

In [ ]:
clustering